### Homework exercises from LLM Zoomcamp Unit 1

In [70]:
import os
import sys
import requests

import openai
from openai import OpenAI
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm
import tiktoken

In [2]:
#os.environ

### Getting the data

In [2]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

### Q1: Run Elasticsearch

In [7]:
es = Elasticsearch("http://localhost:9200")
es.info()

ObjectApiResponse({'name': '0f83ba671176', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'XcswPKRRS3ORlfYTJjG_ew', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

### Q2: Index the data

In [8]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

In [9]:
index_name = "course-questions"
response = es.indices.create(index=index_name, body=index_settings)
response

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [12]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

### Q3: Search

* Use the query "How do I execute a command in a running docker container?"
* Use only the question and text fields
* Give question a boost of 4 (https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-boosting-query.html)
* Use "type": "best_fields"

Determine the score for the top ranking result

In [33]:
query = "How do I execute a command in a running docker container?"

In [43]:
def search(query, index_name="course-questions", max_results=5):

    es = Elasticsearch("http://localhost:9200")
    
    search_query = {
        "size": 1,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }

    response = es.search(index=index_name, body=search_query)
    for hit in response['hits']['hits']:
        doc = hit['_source']
        print(f"Question: {doc['question']}\nAnswer: {doc['text']}\n\n")
        print(f"Score: {hit['_score']}\n")

In [44]:
search(query, index_name="course-questions", max_results=1)

Question: How do I debug a docker container?
Answer: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)


Score: 84.050095



### Q4: Filtering

* Limit the questions to machine-learning-zoomcamp
* Return 3 results

What is the third question returned?

In [45]:
def retrieve_filtered_docs(query, index_name="course-questions", max_results=5):
    
    es = Elasticsearch("http://localhost:9200")
    
    search_query = {
        "size": max_results,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
    
    response = es.search(index=index_name, body=search_query)
    documents = [hit['_source'] for hit in response['hits']['hits']]
    return documents

In [50]:
documents = retrieve_filtered_docs(query, index_name="course-questions", max_results=3)
documents

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

### Q5: Building a prompt

Take the records recturned from Elasticsearch in Q4 and use this template to build the context.  Separate context entries by two line breaks (\n\n)

In [47]:
context_template = """
Q: {question}
A: {text}
""".strip()

In [60]:
def apply_context_template(doc):
    question = doc.get('question', 'No question')
    text = doc.get('text', 'No text')
    return context_template.format(question=question, text=text)

In [61]:
def build_context(docs):
    return "\n\n".join(apply_context_template(doc) for doc in docs)

In [62]:
context = build_context(documents)

Now use the context you just created along with the "How do I execute a command in a running docker container?" question to construct a prompt using the template below:

In [64]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [66]:
user_question = "How do I execute a command in a running docker container?"

In [67]:
prompt = prompt_template.format(question=user_question, context=context)
prompt

'You\'re a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. Use only the facts from the CONTEXT when answering the QUESTION.\n\nQUESTION: How do I execute a command in a running docker container?\n\nCONTEXT:\nQ: How do I debug a docker container?\nA: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)\n\nQ: How do I copy files from my local machine to docker container?\nA: You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_dire

In [68]:
len(prompt)

1462

### Q6: Tokens

Calculate the number of tokens in our query.

In [71]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [74]:
encoding.encode(prompt)

[63842,
 261,
 4165,
 14029,
 29186,
 13,
 30985,
 290,
 150339,
 4122,
 402,
 290,
 31810,
 8099,
 591,
 290,
 40251,
 7862,
 13,
 7649,
 1606,
 290,
 19719,
 591,
 290,
 31810,
 8099,
 1261,
 55959,
 290,
 150339,
 364,
 107036,
 25,
 3253,
 621,
 357,
 15792,
 261,
 6348,
 306,
 261,
 6788,
 62275,
 9282,
 1715,
 10637,
 50738,
 734,
 48,
 25,
 3253,
 621,
 357,
 15199,
 261,
 62275,
 9282,
 3901,
 32,
 25,
 41281,
 290,
 9282,
 3621,
 306,
 25383,
 6766,
 326,
 151187,
 290,
 7251,
 4859,
 11,
 813,
 484,
 480,
 13217,
 261,
 38615,
 6348,
 558,
 68923,
 2461,
 533,
 278,
 2230,
 7962,
 4859,
 38615,
 464,
 3365,
 523,
 3335,
 290,
 9282,
 382,
 4279,
 6788,
 11,
 15792,
 261,
 6348,
 306,
 290,
 4857,
 9282,
 734,
 68923,
 10942,
 350,
 6555,
 290,
 9282,
 26240,
 446,
 68923,
 25398,
 533,
 278,
 464,
 6896,
 26240,
 29,
 38615,
 198,
 6103,
 277,
 10732,
 391,
 79771,
 1029,
 48,
 25,
 3253,
 621,
 357,
 5150,
 6291,
 591,
 922,
 2698,
 7342,
 316,
 62275,
 9282,
 3901,
 32,
 25

In [78]:
len(encoding.encode(prompt))

322

Note: to decode a token back into a word, you can use the decode_single_token_bytes function

In [79]:
encoding.decode_single_token_bytes(62275)

b' docker'

### Bonus: generating the answer

In [80]:
client = OpenAI()

In [83]:
messages = [
        {"role": "system", "content": "You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. Use only the facts from the CONTEXT when answering the QUESTION."},
        {"role": "user", "content": prompt}
    ]

In [87]:
openai_response = client.chat.completions.create(
    model='gpt-4o',
    messages=messages #,
    #max_tokens=150
)

In [92]:
reply = openai_response.choices[0].message.content.strip()
reply

'To execute a command in a running Docker container, follow these steps:\n\n1. Find the container ID by using the command:\n   ```bash\n   docker ps\n   ```\n\n2. Once you have the container ID, you can execute a command in the running container with the following command:\n   ```bash\n   docker exec -it <container-id> <command>\n   ```\n\nFor example, to start a bash session in the container, you would use:\n```bash\ndocker exec -it <container-id> bash\n```'

In [94]:
print("Reply:", reply)

Reply: To execute a command in a running Docker container, follow these steps:

1. Find the container ID by using the command:
   ```bash
   docker ps
   ```

2. Once you have the container ID, you can execute a command in the running container with the following command:
   ```bash
   docker exec -it <container-id> <command>
   ```

For example, to start a bash session in the container, you would use:
```bash
docker exec -it <container-id> bash
```


### Bonus: calculating the costs

Suppose that on average per request we send 150 tokens and receive back 250 tokens.

How much will it cost to run 1000 requests?

You can see the prices here

On June 17, the prices for gpt4o are:

Input: $0.005 / 1K tokens

Output: $0.015 / 1K tokens

You can redo the calculations with the values you got in Q6 and Q7.

In [95]:
openai_response.usage

CompletionUsage(completion_tokens=108, prompt_tokens=365, total_tokens=473)

In [102]:
# number of input tokens
num_input_tokens = (1000 * 322) 

In [103]:
# number of total tokens
num_output_tokens = (1000 * 473)

In [106]:
cost = ((0.005 * num_input_tokens)/1000) + ((0.015 * num_output_tokens)/1000)
print(f'Total cost of running 1,000 requests ${cost}')

Total cost of running 1,000 requests $8.705


Actually the 1000 cancels itseld out